# Evaluation de la qualité des images

## 1. Récupération de métriques sur les images

Il s'agit de calculer un certain nombre de métriques sur l'ensemble de jeu de données pour évaluer la qualité des images. Ces métriques fournissent des informations  sur différents aspects des images, notamment leur netteté, leur luminosité, leur bruit, leur contraste, leur complexité et leur saturation.

Une fois que toutes les métriques ont été calculées pour chaque image dans le jeu de données, elles sont regroupées dans un DataFrame pour une analyse plus approfondie. Ce DataFrame contient les métriques calculées ainsi que le chemin d'accès à chaque image, ce qui facilite la visualisation et l'interprétation des résultats


In [1]:
import os
import glob
import hashlib
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from skimage.measure import shannon_entropy
from skimage import io

In [2]:
# Fonctions de mesure des caractéristiques d'image
def variance_of_laplacian(image):
    """
    Calcul de la variance du Laplacien pour évaluer la netteté de l'image.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    return laplacian.var()

def tenengrad(image):
    """
    Mesure de la netteté de l'image avec les filtres de Sobel.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(sobelx**2 + sobely**2)
    return np.mean(gradient_magnitude)

def brightness(image):
    """
    Mesure de la luminosité moyenne d'une image.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.mean(gray)

def estimate_noise(image):
    """
    Estimation du bruit d'une image.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    noise = gray - blurred
    return np.std(noise)

def snr(image):
    """
    Calcul du rapport signal sur bruit (SNR) d'une image.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean_signal = np.mean(gray)
    noise = gray - cv2.GaussianBlur(gray, (3, 3), 0)
    noise_std = np.std(noise)
    return mean_signal / noise_std

def contraste(image):
    """
    Calcul du contraste d'une image en niveaux de gris.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.std(gray)

def entropie(image_path):
    """
    Calcul de l'entropie de Shannon d'une image pour évaluer sa complexité.
    """
    image = io.imread(image_path, as_gray=True)
    return shannon_entropy(image)

def saturation(image):
    """
    Calcul de la saturation moyenne d'une image pour évaluer la richesse des couleurs.
    """
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    return hsv_image[:, :, 1].mean()

def ratiozoom(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    total_contour_area = sum(cv2.contourArea(contour) for contour in contours)
    image_area = image.shape[0] * image.shape[1]
    
    contour_area_ratio = total_contour_area / image_area
    
    return contour_area_ratio

def densite(image) :
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Détection de contours avec Canny
    edges = cv2.Canny(blurred, 50, 150)
    
    # Trouver les contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Nombre d'objets détectés (contours)
    num_objects = len(contours)
    
    return num_objects

def green_percentage(image):
    """
    Calcule le pourcentage de vert dans une image.
    """
    green_channel = image[:, :, 1]
    green_quantity= np.sum(green_channel)
    color_total = np.sum(image)
    green_percent = (green_quantity / color_total) * 100
    return green_percent

In [3]:
# Chemin du répertoire contenant les images
repertoire = r'D:\aprojet\fev24_cds_plants\data\color'

# Liste pour stocker les informations sur les images
results = []

# Parcours des sous-répertoires et des fichiers
for dossier, sous_dossiers, fichiers in os.walk(repertoire):
    for nom_fichier in glob.glob(os.path.join(dossier, '*.jpg')):
        chemin_image = nom_fichier
        nom_image = os.path.basename(nom_fichier)

        image = cv2.imread(nom_fichier)
        if image is None:
            print(f"Could not read image {nom_fichier}")
            continue

        # Calcul des métriques d'image
        metrics = {
            'image_path': nom_fichier,
            'sharpness': variance_of_laplacian(image),
            'sobel': tenengrad(image),
            'brightness': brightness(image),
            'noise': estimate_noise(image),
            'snr': snr(image),
            'contrast': contraste(image),
            'entropy': entropie(chemin_image),
            'saturation': saturation(image),
            'ratiozoom': ratiozoom(image),
            'densite' : densite(image),
            'pourcentage_vert' : green_percentage(image)
        }

        results.append(metrics)

# Création du DataFrame à partir des résultats
df = pd.DataFrame(results)

# Sauvegarde des métriques dans un fichier CSV
output_csv = 'metriques.csv'
df.to_csv(output_csv, index=False)



In [2]:
# Chargement des métriques depuis le fichier CSV
metriques = pd.read_csv("metriques.csv")

# Affichage des statistiques descriptives
print(metriques.describe())

          sharpness         sobel    brightness         noise           snr  \
count  54303.000000  54303.000000  54303.000000  54303.000000  54303.000000   
mean    3813.553265     67.667131    120.703989    119.651939      1.009356   
std     3576.296836     25.666125     19.509071      3.408372      0.166654   
min        4.659030      8.831932      8.246216     57.438463      0.138736   
25%      910.393366     47.268886    108.755745    118.035927      0.902036   
50%     3061.415518     67.187766    119.878571    120.116819      1.004527   
75%     5780.254361     86.215938    130.888313    122.091958      1.101466   
max    27109.052995    220.185111    235.058762    124.569545      2.124216   

           contrast       entropy    saturation     ratiozoom       densite  \
count  54303.000000  54303.000000  54303.000000  54303.000000  54303.000000   
mean      40.170087     12.320267     69.074008      0.027204     41.591146   
std       11.593207      0.960753     21.476765    

In [5]:
sns.set(style="whitegrid")

# Créer un dossier pour enregistrer les images
output_dir = "histographs"
os.makedirs(output_dir, exist_ok=True)

# Parcourir les colonnes de vos métriques et créer des sous-graphiques
for i, feature in enumerate(metriques.columns[1:], 1):
    plt.figure(figsize=(8, 6))  # Créer une nouvelle figure pour chaque graphique
    sns.histplot(metriques[feature], kde=True)
    
    # Définir la taille du titre
    plt.title(feature, fontsize=20, fontweight='bold')
    
    # Définir la taille des labels des axes
    plt.xlabel(feature, fontsize=16)
    plt.ylabel('Count', fontsize=16)

    # Définir la taille des ticks des axes
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    
    # Ajouter une grille pour améliorer la lisibilité
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    
    # Enregistrer le graphique dans un fichier image
    output_path = os.path.join(output_dir, f"{feature}.png")
    plt.savefig(output_path, bbox_inches='tight')
    
    # Fermer la figure pour libérer la mémoire
    plt.close()

print(f"All graphs have been saved in the '{output_dir}' directory.")

All graphs have been saved in the 'histographs' directory.
